In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import de_core_news_sm
import json
import regex as re

In [23]:
# comedia 
link1 = "http://www.sensacine.com/peliculas/mejores/nota-espectadores/genero-13005/"

# adventure
link1 = "http://www.sensacine.com/peliculas/mejores/nota-espectadores/genero-13001/"

# action
link = "http://www.sensacine.com/peliculas/mejores/nota-espectadores/genero-13025/"

# drama
link1 = "http://www.sensacine.com/peliculas/mejores/nota-espectadores/genero-13008/"


In [ ]:
def get_list_web_links(link, last_page):
    list_web_links =[link]
    for a in range(2,last_page):
        b = link + "/?page=" + str(a)
        list_web_links.append(b)
    return list_web_links

In [5]:
def get_list_film_names_genre(link, last_page):
    list_web_links_genre =[link]
    list_film_names_genre = []
    list_names = []
    for a in range(2, last_page):
        b = link +"/?page=" + str(a)
        list_web_links_genre.append(b)
    list_links = [requests.get(url).content for url in list_web_links_genre] 
    list_soup = [BeautifulSoup(html, 'lxml') for html in list_links]
    list_film_name = [soup.find_all('h2', {'class': 'tt_18 d_inline'}) for soup in list_soup]
    list_film_names_genre = [item for sublist in list_film_name for item in sublist]
    list_genre_film_names = [re.sub('[\W]+', ' ', item.text) for item in list_film_names_genre]
    list_genre_film_names = [item[1:-1] for item in list_genre_film_names]
    return list_genre_film_names

In [6]:
def get_list_web_links_critics_genre(link, last_page):
    list_web_links_genre =[link]
    list_genre_link = []
    list_link = []
    for a in range(2, last_page):
        b = link +"/?page=" + str(a)
        list_web_links_genre.append(b)
    list_links = [requests.get(url).content for url in list_web_links_genre] 
    list_soup = [BeautifulSoup(html, 'lxml') for html in list_links]
    list_films_genre = [soup.find_all('a', {'class': 'no_underline'}) for soup in list_soup]
    list_films_genre = [item for sublist in list_films_genre for item in sublist]
    for r in list_films_genre:
        list_link.append(r.get('href'))
    for l in list_link:
        c = "http://www.sensacine.com" + str(l) + "criticas-espectadores"
        list_genre_link.append(c)
    return list_genre_link

In [7]:
def get_df_stars_genre(list_link_drama, list_names_genre):
    t = [get_list_web_links(link,10) for link in list_link_drama]
    df = pd.DataFrame(t)
    df = df.applymap(lambda x: requests.get(x).content)
    df = df.applymap(lambda x: BeautifulSoup(x, 'lxml'))
    df = df.applymap(lambda x: x.find_all('span',{'class':'stareval-note'}) )
    df['Film_name'] = list_names_genre
    df_film_critics_genre = df.set_index('Film_name').transpose()
    return df_film_critics_genre

In [8]:
def get_df_critics_genre(list_link_drama, list_names_genre):
    t = [get_list_web_links(link,10) for link in list_link_drama]
    df = pd.DataFrame(t)
    df = df.applymap(lambda x: requests.get(x).content)
    df = df.applymap(lambda x: BeautifulSoup(x, 'lxml'))
    df = df.applymap(lambda x: x.find_all('div',{'class':'content-txt review-card-content'}) )
    df['Film_name'] = list_names_genre
    df_film_critics_genre = df.set_index('Film_name').transpose()
    return df_film_critics_genre

In [9]:
def get_df_critics_genre_flaten(df_film_critics_genre, list_names_genre):
    list_critics = [list(df_film_critics_genre[film_name]) for film_name in list_names_genre]
    list_critics_flaten = [[item for sublist in i for item in sublist] for i in list_critics]
    critics = [sorted(set(item), key = item.index) for item in list_critics_flaten]
    df = pd.DataFrame(critics)
    df['Film_name'] = list_names_genre
    df_critics_genre_flaten = df.set_index('Film_name').transpose()
    return df_critics_genre_flaten

In [10]:
def save_evaluation_genre_df(df, genre_name):
    df = df.applymap(lambda x: re.sub('[\n]+', ' ', str(x)))
    df = df.applymap(lambda x: re.sub('<div class="content-txt review-card-content"> ', ' ', str(x)))
    df = df.applymap(lambda x: re.sub('<div>', ' ', str(x)))
    df = df.applymap(lambda x: re.sub('[  ]+', ' ', str(x)))
    df = df.applymap(lambda x: re.sub('[\r]+', ' ', str(x)))
    f = genre_name
    df.to_csv(f'./df_evaluations_genre/{f}', index = False)

In [11]:
def save_evaluation_stars_df(df, genre_name):
    df1 = df.applymap(lambda x: [str(u) for u in x])
    df2 = df1.applymap(lambda x: [re.findall("[0-9].[0-9]", t) for t in x])
    df3 = df2.applymap(lambda x: str(x))
    df4 = df3.applymap(lambda x: re.findall("[0-9].[0-9]", x))
    df5 = df4.applymap(lambda x: [re.sub(',','.',h) for h in x])
    df6 = df5.applymap(lambda x: [float(z) for z in x])
    df7 = df6.applymap(lambda x: x[:-6])
    f = genre_name
    df7.to_csv(f'./df_evaluations_genre/{f}', index = False)

In [19]:
def get_film_name_number(list_web_links, list_film_names):
    t = [re.findall('[0-9]', v) for v in list_web_links]
    q = [("".join(z)) for z in t] 
    dic = dict(zip(q, list_film_names))
    return dic

In [26]:
film_names_drama_sp = get_list_film_names_genre(link, 6)

In [12]:
film_names_comedy_sp = get_list_film_names_genre(link, 6)

In [24]:
film_names_action_sp = get_list_film_names_genre(link, 6)

In [27]:
web_links_critics_drama_sp = get_list_web_links_critics_genre(link, 6)

In [13]:
web_links_critics_comedy_sp = get_list_web_links_critics_genre(link, 6)

In [25]:
web_links_critics_action_sp = get_list_web_links_critics_genre(link, 6)

In [28]:
df_film_critics_drama_sp = get_df_critics_genre(web_links_critics_drama_sp, film_names_drama_sp)

In [14]:
df_film_critics_comedy_sp = get_df_critics_genre(web_links_critics_comedy_sp, film_names_comedy_sp)

In [26]:
df_film_critics_action_sp = get_df_critics_genre(web_links_critics_action_sp, film_names_action_sp)

In [29]:
df_drama_sp = get_df_critics_genre_flaten(df_film_critics_drama_sp, film_names_drama_sp)

In [15]:
df_comedy_sp = get_df_critics_genre_flaten(df_film_critics_comedy_sp, film_names_comedy_sp)

In [27]:
df_action_sp = get_df_critics_genre_flaten(df_film_critics_action_sp, film_names_action_sp)

In [30]:
save_evaluation_genre_df(df_drama_sp, 'drama_sp')

In [16]:
save_evaluation_genre_df(df_comedy_sp, 'comedy_sp')

In [28]:
save_evaluation_genre_df(df_action_sp, 'action_sp')

In [22]:
df_film_stars_drama = get_df_stars_genre(web_links_critics_drama_sp, film_names_drama_sp)

In [17]:
df_film_stars_comedy = get_df_stars_genre(web_links_critics_comedy_sp, film_names_comedy_sp)

In [29]:
df_film_stars_action = get_df_stars_genre(web_links_critics_action_sp, film_names_action_sp)

In [25]:
save_evaluation_stars_df (df_film_stars_drama, 'drama_stars_sp')

In [18]:
save_evaluation_stars_df (df_film_stars_comedy, 'comedy_stars_sp')

In [30]:
save_evaluation_stars_df (df_film_stars_action, 'action_stars_sp')

In [15]:
dic_film_number_drama_sp = get_film_name_number(web_links_critics_drama_sp, film_names_drama_sp)

In [18]:
a_file = open("./dic_film_number/dic_film_number_drama_sp", "w") 
json.dump(dic_film_number_drama_sp, a_file)
a_file.close()

In [20]:
dic_film_number_comedy_sp = get_film_name_number(web_links_critics_comedy_sp, film_names_comedy_sp)

In [21]:
a_file = open("./dic_film_number/dic_film_number_comedy_sp", "w") 
json.dump(dic_film_number_comedy_sp, a_file)
a_file.close()

In [31]:
dic_film_number_action_sp = get_film_name_number(web_links_critics_action_sp, film_names_action_sp)

In [32]:
a_file = open("./dic_film_number/dic_film_number_action_sp", "w") 
json.dump(dic_film_number_action_sp, a_file)
a_file.close()